In [0]:
import pandas as pd

file_path = "/Volumes/workspace/sales_data/sales_data/sales_data.csv"

df = pd.read_csv(file_path)

df.head()


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


Standardize column names

In [0]:
# make column names python-friendly
df.columns = (
    df.columns
    .str.lower()
    .str.replace(" ", "_")
)

df.columns


Index(['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode',
       'customer_id', 'customer_name', 'segment', 'country', 'city', 'state',
       'postal_code', 'region', 'product_id', 'category', 'sub-category',
       'product_name', 'sales'],
      dtype='object')

Select only the columns we need

In [0]:
df = df[
    [
        "order_date",
        "region",
        "category",
        "sales"
    ]
]

df.head()


,order_date,region,category,sales
0,08/11/2017,South,Furniture,261.9600
1,08/11/2017,South,Furniture,731.9400
2,12/06/2017,West,Office Supplies,14.6200
3,11/10/2016,South,Furniture,957.5775
4,11/10/2016,South,Office Supplies,22.3680


Convert order_date to datetime

In [0]:
df["order_date"] = pd.to_datetime(df["order_date"], dayfirst=True)


In [0]:
df.dtypes


order_date    datetime64[ns]
region                object
category              object
sales                float64
dtype: object

In [0]:
summary = {
    "total_sales": float(df["sales"].sum()),
    "average_sales": float(df["sales"].mean()),
    "sales_by_region": df.groupby("region")["sales"].sum().to_dict(),
    "sales_by_category": df.groupby("category")["sales"].sum().to_dict(),
    "monthly_sales_trend": (
        df
        .groupby(df["order_date"].dt.month)["sales"]
        .sum()
        .to_dict()
    )
}

summary


{'total_sales': 2261536.7827000003,
 'average_sales': 230.7690594591837,
 'sales_by_region': {'Central': 492646.9132,
  'East': 669518.726,
  'South': 389151.459,
  'West': 710219.6845},
 'sales_by_category': {'Furniture': 728658.5757,
  'Office Supplies': 705422.334,
  'Technology': 827455.873},
 'monthly_sales_trend': {1: 94291.6296,
  2: 59371.1154,
  3: 197573.5872,
  4: 136283.0006,
  5: 154086.7237,
  6: 145837.5233,
  7: 145535.689,
  8: 157315.927,
  9: 300103.4117,
  10: 199496.2947,
  11: 350161.711,
  12: 321480.1695}}

In [0]:
def build_agent_prompt(question, summary):
    return f"""
You are a business analyst.

Here is the summarized sales data:
- Total Sales: {summary['total_sales']}
- Average Sales: {summary['average_sales']}
- Sales by Region: {summary['sales_by_region']}
- Sales by Category: {summary['sales_by_category']}
- Monthly Sales Trend: {summary['monthly_sales_trend']}

User question:
{question}

Explain trends, anomalies, and business insights in simple, clear language.
"""


In [0]:
test_prompt = build_agent_prompt(
    "Why did sales decline in certain months?",
    summary
)

print(test_prompt)



You are a business analyst.

Here is the summarized sales data:
- Total Sales: 2261536.7827000003
- Average Sales: 230.7690594591837
- Sales by Region: {'Central': 492646.9132, 'East': 669518.726, 'South': 389151.459, 'West': 710219.6845}
- Sales by Category: {'Furniture': 728658.5757, 'Office Supplies': 705422.334, 'Technology': 827455.873}
- Monthly Sales Trend: {1: 94291.6296, 2: 59371.1154, 3: 197573.5872, 4: 136283.0006, 5: 154086.7237, 6: 145837.5233, 7: 145535.689, 8: 157315.927, 9: 300103.4117, 10: 199496.2947, 11: 350161.711, 12: 321480.1695}

User question:
Why did sales decline in certain months?

Explain trends, anomalies, and business insights in simple, clear language.



In [0]:
with open("/Volumes/workspace/sales_data/sales_data/agent_prompt.txt", "w") as f:
    f.write(test_prompt)

print("Prompt saved.")


Prompt saved.
